In [15]:
import torch
from transformers import AutoTokenizer, BertForSequenceClassification
from datasets import load_dataset
import pandas as pd
from tqdm import tqdm
import click
tokenizer = AutoTokenizer.from_pretrained("ProsusAI/finbert")
# model = BertForSequenceClassification.from_pretrained("ProsusAI/finbert")


In [2]:
dataset= load_dataset('sehyun66/Finnhub-News','clean',split='clean')
df = pd.DataFrame(dataset)

Generating clean split: 100%|██████████| 316086/316086 [00:00<00:00, 352439.51 examples/s]


In [12]:
dataset['new_cloumns'] = dataset.map

TypeError: 'Dataset' object does not support item assignment

In [13]:
texts=list(df['headline'])

In [33]:
headline_inputs = tokenizer(texts, padding=True ,truncation=True,return_tensors="pt")
summary_inputs = tokenizer(texts, padding=True ,truncation=True,return_tensors="pt")


In [36]:
import sys
sys.getsizeof(headline_inputs)
sys.getsizeof(summary_inputs)



48

In [37]:
dataset.to_json(f"my-dataset-{}.jsonl")

48

In [39]:
x= [1,1,1,1]
sys.getsizeof(x)

88

In [9]:
with torch.no_grad():
    logits = model(**inputs).logits

predicted_class_id = logits.argmax().item()
print(model.config.id2label[predicted_class_id])


In [10]:
device = torch.device('cuda')

In [26]:
def encoding(texts):
    chunk_size= 1024
    results=[]
    for i in tqdm(range(0, len(texts), chunk_size)):
        chunk = texts[i:i+chunk_size]
        output = tokenizer(texts, padding=True ,truncation=True,return_tensors="pt")
        results.extend(output['input_ids'])
    return results

In [29]:
dataset.features

{'datetime': Value(dtype='int64', id=None),
 'image': Value(dtype='string', id=None),
 'related': Value(dtype='string', id=None),
 'source': Value(dtype='string', id=None),
 'summary': Value(dtype='string', id=None),
 'url': Value(dtype='string', id=None),
 'id': Value(dtype='int64', id=None),
 'category': Value(dtype='string', id=None),
 'headline': Value(dtype='string', id=None)}

In [ ]:
model.to(device)
model = torch.nn.DataParallel(model)
inputs.to(device)

In [39]:
# Define the chunk size
chunk_size = 1024  # Adjust this value as needed

# Initialize an empty list to store the results
results = []

# Loop through the input data in chunks
for i in tqdm(range(0, len(inputs['input_ids']), chunk_size)):
    chunk = inputs['input_ids'][i:i+chunk_size]
    chunk.to(device)
    # Perform inference on the chunk
    with torch.no_grad():
        output = model(chunk).logits   
    # Append the output to the results list
    results.append(output)

# Concatenate or process the results as needed
final_output = torch.cat(results, dim=0)  # Concatenate along the appropriate dimension


100%|██████████| 504/504 [04:37<00:00,  1.82it/s]


In [40]:
torch.save(final_output, 'sentiment_data/finbert.pt')

In [44]:
final_output[0]

tensor([-0.6055, -0.4091,  0.8146], device='cuda:0')

In [52]:
df['sentiment'] = final_output.to('cpu').tolist()
df['sentiment_name'] = df['sentiment'].apply(lambda x:{'postive':x[0],'negative':x[1],'neutral':x[2]})

In [67]:
df['sentiment_name'] = df['sentiment'].apply(lambda x:{'postive':x[0],'negative':x[1],'neutral':x[2]})

In [76]:
file_path = 'sentiment_data/finbert.json'
json_data = df.to_json(orient='records')
with open(file_path, 'w') as json_file:
    json_file.write(json_data)

In [10]:
clean = load_dataset(
        "json",
        data_files={
            "clean":"/home/ssu36/tiger/NH_competition/data/clean_data/*.json"
        },
        # split ={"train","clean"}
    )
train = load_dataset(
    "json",
        data_files={
            "train": "/home/ssu36/tiger/NH_competition/sentiment_data/*.json",
        },
)

In [11]:
from datasets import DatasetDict

ddict = DatasetDict({
    "train": train,   # split1_ds is an instance of `datasets.Dataset`
    "clean": clean
})

In [13]:
clean.push_to_hub('sehyun66/Finnhub-News','clean')

Pushing dataset shards to the dataset hub: 100%|██████████| 1/1 [00:45<00:00, 45.92s/it]
